# Handling Multiple Tables

Adapted from: https://learning.oreilly.com/library/view/pandas-in-action/9781617297434/OEBPS/Text/10.htm 

This Notebook covers:

* Concatenating DataFrames on the vertical and horizontal axes

* Merging DataFrames with inner joins, outer joins, and left joins

* Joining DataFrames by index labels


# Introducing the Data Sets

Data sets come from the online social service __Meetup__, a website where users join groups for common interests such as hiking, literature, and board games. 

Group organizers schedule remote or in-person events that group members attend. 

Meetup’s domain has several data models, including groups, categories, and cities.

The meetup directory houses all data sets for this chapter.

In [ ]:
import pandas as pd

### Registered groups: groups1.csv and groups2.csv

Let’s begin our exploration by importing the groups1.csv and groups2.csv files. 

These files hold a __sample of Meetup’s registered groups__. 

Each group includes an group ID, name, associated category ID, and associated city ID.

In [ ]:
groups1 = pd.read_csv("meetup/groups1.csv")
print(groups1.head())
print(groups1.shape)


In [ ]:
groups2 = pd.read_csv("meetup\groups2.csv")
print(groups2.head())
print(groups2.shape)

Notice that both CSVs have the same four columns. 

The groups data was somehow split and stored across two files instead of one.

### Information about Categories :  categories.csv

Each group has a category_id  key. 

We can find information on categories in the categories.csv file. 

Each row in this file stores the category’s ID and name:

In [ ]:
categories = pd.read_csv("meetup\categories.csv")
print(categories.head())
print(categories.shape)  # we have 27 categories

### Information about cities : cities.csv

Each group also has a city_id foreign key. 

The cities.csv data set stores the city information. 

A city has a unique ID, name, state, and zip code. 

In [ ]:
pd.read_csv("meetup\cities.csv").head()

The cities data set has a small issue. 

Look at the zip value in the first row. 7093 is an invalid zip code; the value in the CSV is in fact 07093. 

Zip codes can start with a leading zero. Unfortunately, pandas assumes that the zip codes are integers and thus strips the leading zeroes from the values. 

To solve this problem, we can add the __dtype parameter__ to the __read_csv__ method. 

dtype accepts a dictionary in which keys denote column names and values denote the data type to assign to that column.

In [ ]:
cities = pd.read_csv(
    "meetup\cities.csv", dtype = {"zip": "string"}
)
print(cities.dtypes)
cities.head()


# Concatenating Data Sets

The simplest way to combine two data sets is with __concatenation__ — __appending one DataFrame to the end of another__.

 ### concatenating groups1 and groups2

The groups1 and groups2 DataFrames both have the same four column names. 
 They are two halves of a greater whole. 
 
 To combine their rows into a single DataFrame,  Pandas has a convenient __concat__ method at the top level of the library. 
 
 We can pass a list of DataFrames. Pandas will concatenate the objects in the order in which they appear in the objs list.

Next example concatenates the rows in groups2 to the end of groups1:

In [ ]:
groups = pd.concat(objs = [groups1, groups2])
print(groups1.shape)
print(groups2.shape)
print(groups.shape)
groups


The concatenated DataFrame has 16,330 rows! 

Its length is equal to the sum of the lengths of the groups1 and groups2

In [ ]:
len(groups1)

In [ ]:
len(groups2)

In [ ]:
len(groups1) + len(groups2)

Pandas preserves the original index labels from both DataFrames in the concatenation, which is why we see a final index position of 8,330 in the concatenated DataFrame even though it has more than 16,000 rows. 

What we are seeing is the 8,330 index from the end of the groups2 DataFrame. 

Pandas does not care that the same index number appears in both groups1 and groups2. As a result, the concatenated index has duplicate index labels.

We can pass the concat function’s __ignore_index parameter__ an argument of True to generate pandas’ standard numeric index. 

The concatenated DataFrame will discard the original index labels:

In [ ]:
pd.concat(objs = [groups1, groups2], ignore_index = True)

To create a nonduplicate index but also preserve which DataFrame each row of data came from one solution is to add a __keys parameter__ and __pass it a list of strings__. 

Pandas will associate each string in the keys list with the DataFrame at the same index position in the objs list. 

The keys and objs lists must be of equal length.

The next example assigns the groups1 DataFrame a key of "G1" and the groups2 DataFrame a key of "G2". 

The concat function returns a MultiIndex DataFrame. 

The MultiIndex’s first level stores the keys, and its second level stores the index labels from the respective DataFrame:

In [ ]:
pd.concat(objs = [groups1, groups2], keys = ["G1", "G2"])

To end this section, let’s assign the concatenated DataFrame to a groups variable

In [ ]:
groups = pd.concat(objs = [groups1, groups2], ignore_index = True)
groups.head()

##  Missing Values in Combined DataFrames

When concatenating two DataFrames, pandas places __NaNs__ at intersections of row labels and column labels that the data sets do not share. 

Consider the following two DataFrames, both of which have a Football column. 

The sports_champions_A DataFrame has an exclusive Baseball column, and the sports_champions_B DataFrame has an exclusive Hockey column.

In [ ]:
sports_champions_A = pd.DataFrame(
    data = [
        ["New England Patriots", "Houston Astros"],
        ["Philadelphia Eagles", "Boston Red Sox"]
    ],
    columns = ["Football", "Baseball"],
    index = [2017, 2018]
)

sports_champions_A

In [ ]:
sports_champions_B = pd.DataFrame(
    data = [
        ["New England Patriots", "St. Louis Blues"],
        ["Kansas City Chiefs", "Tampa Bay Lightning"]
    ],
    columns = ["Football", "Hockey"],
    index = [2019, 2020]
)

If we concatenate the DataFrames, we will create missing values in the Baseball and Hockey columns. 

The sports_champions_A DataFrame has no values to place in the Hockey column, and the sports_champions_B DataFrame has no values to place in the Baseball column:

In [ ]:
pd.concat(objs = [sports_champions_A, sports_champions_B])

# Joins

Compared with a concatenation, __a join uses a logical criterion to determine which rows or columns to merge between two data sets__. 

For examples, a join can target only rows with shared values between both data sets. 


The following sections cover three types of joins: left, inner, and outer. Let’s walk through them one by one.


## Left Joins

A left join __uses keys from one data set to pull in values from another__. 

It is equivalent to a __VLOOKUP operation in Excel__. 

A left join is optimal when one data set is the focal point of the analysis. We __pull in the second data set to provide supplemental information related to the primary data set__. 


<img src="files/CH10_F01_Paskhaver.png" width="800" height="400">

![](./CH10_F01_Paskhaver.png)

Here’s a quick reminder of what our groups and categories data sets look like

In [ ]:
groups.head(3)

In [ ]:
categories.head(3)

Let’s execute a left join on groups to __add category information for each group__. 

We’ll use the __merge method__ to merge one DataFrame into another. 

The method’s first parameter, __right__, accepts a DataFrame. The terminology comes from the previous diagram. 

The right DataFrame is  the “second” data set. 

We can pass a string denoting the type of join to the method’s __how parameter__. In our case is  "left". 

We also must tell pandas which columns to use to match values between the two DataFrames. Let’s add an __on parameter__ with a value of "category_id". 

We can use the on parameter only when the column name is equal between DataFrames. In our case, both the groups and categories DataFrames have a category_id column:

In [ ]:
groups.merge(categories, how = "left", on = "category_id").head()

Now we have an aditional column, with category name.

Pandas pulls in the categories table’s columns whenever it finds a match for the category_id value in groups. 
The one exception is the category_id column, which is listed only once. 

Note that when the library does not find a category_id in categories, it displays __NaN values__ in the category_name column from categories. We can see an example on rows 2 and 4 of the previous output.

## Inner Joins

An inner join targets values that exist across two DataFrames.

In an inner join, pandas excludes values that exist only in the first DataFrame and only in the second DataFrame.

In [ ]:
groups.head(3)

In [ ]:
categories.head(3)

Let’s identify the categories that exist in both data sets. 

From a technical perspective, we once again want to target the rows from the two DataFrames with equal values in the category_id columns. 
In this situation, it doesn’t matter whether we invoke the merge method on group or categories. 

An inner join identifies common elements in both data sets; the results will be the same regardless. 

For the next example, let’s call the merge method on groups:

In [ ]:
groups.merge(categories, how = "inner", on = "category_id")


Let’s look  at what pandas did. 

The first four rows in the merged DataFrame have a category_id of 14. 

We can filter for that ID in the groups and categories DataFrames:

In [ ]:
groups[groups["category_id"] == 14]

In [ ]:
categories[categories["category_id"] == 14]

The merged DataFrame creates one row for each group_id match across the two DataFrames. 

There are 870 rows in groups and one row in categories with a group_id of 14. 

Pandas pairs each of the 870 rows in groups with the single row in categories and creates a total of 870 rows in the merged DataFrame. 

Because an inner join creates a new row for each value match, the merged DataFrame can be significantly larger than the original ones. If there were three categories with an ID of 14, for example, pandas would create 2610 rows (870 x 3).

## Outer Joins

An outer join combines all records across two data sets. 

__Exclusivity does not matter__ with an outer join. 

pandas includes all values irrespective of whether they belong in one data set or both data sets.

In [ ]:
groups.head(3)

In [ ]:
cities.head(3)

Let’s merge groups and cities with an outer join. 

We’ll pull in all cities: the ones exclusive to groups, the ones exclusive to cities, and the ones common to both.

So far, we’ve used only shared column names to merge data sets. 

__When column names differ between data sets__, we must pass different parameters to the merge method. 

Instead of the on parameter, we can use the merge method’s __left_on__ and __right_on__ parameters. 

We pass left_on the column name in the left DataFrame and right_on the column name in the right DataFrame. 

In [ ]:
groups.merge(
    cities, how = "outer", left_on = "city_id", right_on = "id"
)

The final DataFrame has all city IDs from both data sets. 

If pandas finds a values match between the city_id and id columns, it merges the columns from the two DataFrames in a single row. 

We can see some examples in the first five rows. The city_id column stores the common id.

We can pass True to the merge method’s __indicator parameter__ to identify which DataFrame a value belongs to.

The merged DataFrame will include a _merge column that stores the values "both", "left_only", and "right_only"

In [ ]:
groups.merge(
    cities,
    how = "outer",
    left_on = "city_id",
    right_on = "id",
    indicator = True
)

We can use the _merge column to filter rows that belong to either of the DataFrames. 

The next example extracts rows with a value of "right_only" in the _merge column or, equivalently, the city IDs that are present only in cities

In [ ]:
outer_join = groups.merge(
    cities,
    how = "outer",
    left_on = "city_id",
    right_on = "id",
    indicator = True
)

in_right_only = outer_join["_merge"] == "right_only"

outer_join[in_right_only].head()

# Merging on Index Labels

Imagine that a DataFrame we’d like to join stores its primary keys in its index.

Let’s simulate this scenario. 

We can invoke the set_index method on cities to set its id column as its DataFrame index:

In [ ]:
cities.head(3)

In [ ]:
cities = cities.set_index("id")

In [ ]:
cities.head(3)

In [ ]:
groups.head(3)

Let’s use a left join to merge cities into groups again

To look for matches in the index of the right DataFrame, we can provide a different parameter, __right_index__, and set it to True. 

The argument tells pandas to look for city_id matches in the right DataFrame’s index

In [ ]:
groups.merge(
    cities,
    how = "left",
    left_on = "city_id",
    right_index = True
)

# Coding Challenge

Let’s practice the concepts introduced ...

We will use sales in a fictional restaurant. 

The week_1_sales.csv and week_2_sales.csv files hold listings of weekly transactions.

 Each restaurant order includes the ID of a customer who placed an order and the ID of the food item they purchased. 
 
 Here’s a preview of the first five rows of week_1_sales:

In [ ]:
pd.read_csv("restaurant/week_1_sales.csv").head()

The week_2_sales data set has an identical shape. 

Let’s import the two CSVs and assign them to week1 and week2 variables:

In [ ]:
week1 = pd.read_csv("restaurant/week_1_sales.csv")
week2 = pd.read_csv("restaurant/week_2_sales.csv")

The Customer ID columns hold foreign keys that reference values in the ID column in customers.csv. 

Each record in customers.csv includes a customer’s first name, last name, gender, company, and occupation. 

Let’s import that data set with the read_csv function and set its ID column as the DataFrame index with the index_col parameter:

In [ ]:
pd.read_csv("restaurant/customers.csv", index_col = "ID").head()

In [ ]:
customers = pd.read_csv(
    "restaurant/customers.csv", index_col = "ID"
)

There’s another column of  keys in the weeks1 and weeks2 DataFrames. 

The Food ID  key connects to the ID column in foods.csv. 

A food item includes an ID, a name, and a price. When we import this data set, let’s set its Food ID column as the DataFrame index.

In [ ]:
pd.read_csv("restaurant/foods.csv", index_col = "Food ID")

In [ ]:
foods = pd.read_csv("restaurant/foods.csv", index_col = "Food ID")

## Problems

 
1. Concatenate the two weeks of sales data into one DataFrame. Assign the week1 DataFrame a key of "Week 1" and the week2 DataFrame a key of "Week 2".

2. Find the customers who ate at the restaurant both weeks.

3. Find the customers who ate at the restaurant both weeks and ordered the same item each week.

* HINT You can join data sets on multiple columns by passing the on parameter a list of columns.

4. Identify which customers came in only on Week 1 and only on Week 2.

5. Each row in the week1 DataFrame identifies a customer who purchased a food item. For each row, pull in the customer’s information from the customers DataFrame.

 

## Solutions

### 1 - Concatenate the two weeks of sales data into one DataFrame. Assign the week1 DataFrame a key of "Week 1" and the week2 DataFrame a key of "Week 2".

##### Resolution:

In [ ]:
pd.concat(objs = [week1, week2], keys = ["Week 1", "Week 2"])

### 2 - Find the customers who ate at the restaurant both weeks.

##### Resolution:

In [ ]:
week1.merge(
    right = week2, how = "inner", on = "Customer ID"
).head()

In [ ]:
week1.merge(
    right = week2, how = "inner", on = "Customer ID"
).drop_duplicates(subset = ["Customer ID"]).head()

### 3 -  Find the customers who ate at the restaurant both weeks and ordered the same item each week.

##### Resolution:

In [ ]:
week1.merge(
    right = week2,
    how = "inner",
    on = ["Customer ID", "Food ID"]
)

### 4 - Identify which customers came in only on Week 1 and only on Week 2.

##### Resolution:

In [ ]:
week1.merge(
    right = week2,
    how = "outer",
    on = "Customer ID",
    indicator = True
).head()

### 5 - Each row in the week1 DataFrame identifies a customer who purchased a food item. For each row, pull in the customer’s information from the customers DataFrame.

##### Resolution:

In [ ]:
week1.merge(
    right = customers,
    how = "left",
    left_on = "Customer ID",
    right_index = True
).head()